### Portfolio Testing (pricing)

In [101]:
import pandas as pd
import numpy as np
from environment import Environment

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [102]:
from historical import HistoricalData
from pandas._libs.tslibs.timestamps import Timestamp, Timedelta

path = "data\\"
env_df = pd.read_csv(path + "Consolidated.csv", header = [0,1], index_col=[0])
targets_df = pd.read_csv(path + "Targets.csv", index_col=[0])
features_df = pd.read_csv(path + "Features.csv", header = [0,1], index_col=[0])


data = HistoricalData(env_df, features_df, targets_df)

In [103]:
import instrument
i1 = instrument.Equity('SPY US Equity', 'USD')
i2 = instrument.Option(name='SPY Put', ccy='USD', is_call=0, ul='SPY US Equity', K=295, T=Timestamp('2017-03-31'))
date = Timestamp('2017-01-31')
prices, fx, divs, surfaces, curves = data.get_env_args(date)
env_now = Environment(date, prices, fx, divs, surfaces, curves)

In [104]:
i2.value(env_now)

4.892897657438808

In [105]:
from portfolio import Portfolio
my_pf = Portfolio({i1: 2, i2:-1})

In [106]:
my_pf.calc_value(env_now)

586.9967023425612

### Regression Testing

In [140]:
from historical import ShockMap, Distribution
sm = ShockMap(data, date)
dist = Distribution(data, date)

In [141]:
for i in range(10):
    asset_shocks = sm.map_factors(dist.generate_shock())
    env_sim = env_now.simulate(asset_shocks)
    print(my_pf.calc_value(env_sim))

533.9476335551182
589.4469943070443
573.7148468047685
602.1543098515525
567.6523065865786
594.587060264492
618.7439918600419
607.6052465581005
571.7914773406378
595.9068818189423
